In [1]:
%run Functions.py

In [2]:
def LettersNotSelected(UserKeys, PhrasesStim, PhraseUser):
    
    UserKeysTime = [val[0] for val in UserKeys]
    
    ansr = list()

    
    
    # find starting point of every phrase 
    for ind in range(0,len(PhrasesStim)):
        print(ind+1, PhrasesStim[ind][1])
        timePhraseStarts = PhrasesStim[ind][0]
        
        # Find the time when the phrase starts
        timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePhraseStarts)
        
        
        # find when the phrase ends:
        if ind == len(PhrasesStim)-1: # if it is the last sentence, reverse index searching starts from the last index
            ind_end = len(UserKeys) - 1
        else: # if it's not the last sentence, reverse index searching starts from the next phrase
            ind_end_time, ind_end = nearestTimePoint(UserKeysTime, PhrasesStim[ind+1][0])
         
        
        flag_start = 0
        typing_list = list()
        letter_list = list()
        
        
        # collect the typing letters in a list of lists
        for ind_typing in range(ind_start, ind_end):
            
            if flag_start > 0:
                letter_list.append(UserKeys[ind_typing]) 
                if UserKeys[ind_typing][1] != UserKeys[ind_typing+1][1]:
                    typing_list.append(letter_list)
                    letter_list = list()
            else:
                letter_list.append(UserKeys[ind_typing])
                flag_start = 1
          
        
        
        # from the collected letters, find the ones that are not selected
        typing_list_temp = list()
        
        ind_letter = -1
        for letter in typing_list:
            ind_letter = ind_letter + 1
            typing_list_temp.append(letter[0][1])
            for instance in letter:
                if int(instance[2]) == 1:
                    if len(typing_list_temp) > 2 and letter[0][1] == typing_list_temp[-3] and letter[0][2] > typing_list[ind_letter-2][-1][2]:
                        del typing_list_temp[-3]
                        
                    del typing_list_temp[-1]
                    
                    break
        
        print(PhraseUser[ind][1])
        
        
        ansr.append(len(typing_list_temp)/len(PhraseUser[ind][1]))
        
        
    
    return ansr

In [3]:
metricComputed = 'AttendedButNotSelected'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'
resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\ls2_4' +  '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
        if 'Picture' in root:
            continue
        if 'ls2\\4\\' not in root:
            continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # find the language of the data:
            if 'Hvad er det fulde navn på dit universitet?' in phraseLog[1][1]: 
                language = 'Danish'
            else:
                language = 'English'
                
                
            phraseStim_reduced = stimPhrasesEdit(phraseLog_new, root)
            
            # if it is the 2nd part of the session, picture is not described:
            if '2ndPart' in root:
                picture = 'not_described'
            else:
                picture = 'described'
            
            phraseUserEnd_reduced = scratchPadPhraseEdit(scratchPad_new, subjName, root, picture)
            
            # Find letters not selected:
            # Find all keys that are not selected, just shortly looked at 
            ansr = LettersNotSelected(userKeys_wDwellTime, phraseStim_reduced, phraseUserEnd_reduced)
            
            if '1stPart' in root:
                print('1stPart')
                ansr1 = ansr 
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                ansr2 = ansr
                
                ansr = ansr1 + ansr2
                
                ansr1 = list()
                
                
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = ansr
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            #dataToSave.AddToFile()
            

subject path E:\Data\Data\ls2\4\2019-02-18-10-09-22_1
subject and session name:  ls2__4__2019-02-18-10-09-22_1
1 Sæt fadet i ovnen i ca. 30 min.
Sæt fadet i ovnen i ca. 30 min.
2 Aldrig har jeg set et hold brænde så mange store chancer.
Aldrig har jeg set et hold brænde så mange store chancer.
3 Rengør med varmt vand og tør grundigt af.
Rengør med varmt vand og tør grundigt af.
4 Slut Mac mini til fjernsynet for at se dvd'er eller film.
Slut Mac mini til fjernsynet for at se dvd'er eller film.
5 Af udvalget må der vælges 2 slags stege,  - hvis ekstra steg ønskes koster det kr 20, - pr. person.
Af udvalget må der vælges to slags stege,  - hvis ekstra steg ønskes koster det 20 kr. pr. person.
  subjectID block session  trial  AttendedButNotSelected
0       ls2     4       1      0                0.709677
1       ls2     4       1      1                0.649123
2       ls2     4       1      2                0.658537
3       ls2     4       1      3                0.824561
4       ls2    